In [1]:
import pandas as pd
from datetime import datetime

df = pd.read_csv("cleaned_eta_logs.csv")
df["datetime"] = pd.to_datetime(df["datetime"])

baseline_start = datetime(2025, 11, 6, 21, 30)
baseline_end   = datetime(2025, 11, 6, 22, 0)
peak_start     = datetime(2025, 11, 6, 22, 20)
peak_end       = datetime(2025, 11, 6, 23, 10)

baseline = df[(df["datetime"] >= baseline_start) & (df["datetime"] < baseline_end)]
peak     = df[(df["datetime"] >= peak_start)   & (df["datetime"] < peak_end)]

def stats(s):
    return {
        "count": len(s),
        "mean": s["execution_time"].mean(),
        "median": s["execution_time"].median(),
        "p95": s["execution_time"].quantile(0.95),
        "std": s["execution_time"].std(ddof=0),  # population std to match my numbers
        "min": s["execution_time"].min(),
        "max": s["execution_time"].max(),
        "slow_pct_gt20": (s["execution_time"] > 20).mean() * 100,
        "critical_pct_gt60": (s["execution_time"] > 60).mean() * 100,
    }

b, p = stats(baseline), stats(peak)
deg_factor = p["mean"] / b["mean"]
deg_pct = (p["mean"] - b["mean"]) / b["mean"] * 100

print("Baseline:", b)
print("Peak:", p)
print(f"Slowdown factor (mean): {deg_factor:.2f}x; Increase: {deg_pct:.2f}%")

Baseline: {'count': 2756, 'mean': np.float64(2.0268011611030476), 'median': np.float64(0.901), 'p95': np.float64(7.3420000000000005), 'std': np.float64(2.879717509468294), 'min': np.float64(0.027), 'max': np.float64(34.083), 'slow_pct_gt20': np.float64(0.39912917271407833), 'critical_pct_gt60': np.float64(0.0)}
Peak: {'count': 4708, 'mean': np.float64(13.186807986406118), 'median': np.float64(11.792), 'p95': np.float64(35.74714999999994), 'std': np.float64(13.158711419893468), 'min': np.float64(0.021), 'max': np.float64(115.895), 'slow_pct_gt20': np.float64(16.90739167374681), 'critical_pct_gt60': np.float64(1.6992353440951573)}
Slowdown factor (mean): 6.51x; Increase: 550.62%
